In [1]:
import pandas as pd
from auxiliary.filter_functions import *
from auxiliary.search_algorithm import *
from results import *
from testing import *
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad, SimbadClass
import os
import glob
from astropy.table import Table
import numpy as np
from astropy.io import fits
import time
import matplotlib.pyplot as plt
from astropy.stats import poisson_conf_interval
import scipy
import re

In [ ]:
# Get the candidates with no matches within a certain date range

date_range = [
    # '',
    # '2015-01-01',
    '2022-04-01',
    ''
]

no_matches = get_no_match_fxts(20.0, from_date=date_range[0], to_date=date_range[1])
no_matches

In [ ]:
# plot the light curves of the candidates with no matches
for i, fxt in no_matches.iterrows():
    pass
    # lc = gen_light_curve(
    #     fxt['ObsId'],
    #     float(fxt['RA']),
    #     float(fxt['DEC']),
    #     float(fxt['THETA']),
    # )
    # print(lc)
    # plot_light_curve(
    #     fxt['ObsId'],
    #     float(fxt['RA']),
    #     float(fxt['DEC']),
    #     float(fxt['THETA']),
    # )

In [ ]:
# get the criteria table for a certain date range like in the papers

date_range = [
    # '',
    # '2015-01-01',
    '2022-04-01',
    ''
]

criteria_table = get_criteria_table(from_date=date_range[0], to_date=date_range[1])

criteria_table

In [ ]:
# get table of number of observations, analysed observations, detections and no match detections

date_range = [
    # '',
    '2015-01-01',
    '2022-04-01',
    # ''
]

candidate_numbers = get_candidate_numbers(
    from_date=date_range[0], to_date=date_range[1])

candidate_numbers

In [ ]:
# generate and save random light curves to manually check detections

# random_detections, random_light_curves = get_random_light_curves(50, from_date='', to_date='')
# plot_random_light_curves(random_detections, random_light_curves)

In [ ]:
# test a filter function on a given detection

detection = pd.DataFrame({
    'RA': 187.66173433407874,
    'DEC': 41.6360916527896,
    'POS_ERR': 0.08753291125732836
}, index=[0])

print(detection)

filter_Xray_binaries(detection.iloc[0], True)

In [ ]:
# test search algorithm for a single source

# obsid = 8490
# detections = get_detections(
#     str(obsid),
#     ra='',
#     dec=''
# )[0:1]

# or manually give source
# to check fxt in paper for example

detections = pd.DataFrame({
    'ObsId': ['24604'],
    'RA': ['207.23523'],
    'DEC': ['26.66230'],
    'THETA': ['7.08']
})

for index, detection in detections.iterrows():
    test_search_algorithm(
        detection['ObsId'],
        float(detection['RA']),
        float(detection['DEC']),
        float(detection['THETA'])
    )
    plot_light_curve(
        detection['ObsId'],
        float(detection['RA']),
        float(detection['DEC']),
        float(detection['THETA'])
    )

In [ ]:
# check if fxts from papers are found in new code
paper_I_fxts = [
    [803, 186.38125, 13.06607],
    [2025, 167.86792, 55.67253],
    [8490, 201.24329, -43.04060],
    [9546, 211.25113, 53.65706],
    [9548, 170.07296, 12.97189],
    [14904, 345.49250, 15.94871],
    [4062, 76.77817, -31.86980],
    [5885, 318.12646, -63.49914],
    [9841, 175.00504, -31.91743],
    [12264, 90.00450, -52.71501],
    [12884, 212.12063, -27.05784],
    [13454, 15.93558, -21.81272],
    [15113, 45.26725, -77.88095],
    [16454, 53.16158, -27.85940],
]

paper_II_fxts = [
    [16093, 233.73496, 23.46849],
    [16453, 53.07672, -27.87345],
    [18715, 40.82972, 32.32390],
    [19310, 36.71489, -1.08317],
    [20635, 356.26437, -42.64494],
    [21831, 207.34711, 26.58421 ],
    [23103, 50.47516, 41.24704],
    [24604, 207.23523, 26.66230],
]

# paper_I_fxts = pd.DataFrame(paper_I_fxts, columns=['ObsId', 'RA', 'DEC'])
# paper_II_fxts = pd.DataFrame(paper_II_fxts, columns=['ObsId', 'RA', 'DEC'])

analysed = pd.read_csv('output/analysed_w20.txt', header=0, sep=' ', dtype=str)

print('Paper I FXTs')
for fxt in paper_I_fxts:
    detections = get_detections(str(fxt[0]), fxt[1], fxt[2], True)
    if not detections.empty:
        print(detections[['ObsId', 'RA', 'DEC']].to_string(index=False, header=False))
    elif not analysed[analysed['ObsId'] == str(fxt[0])].empty:
        print(f'{fxt[0]} - not detected')
    else:
        print(f'{fxt[0]} - not analysed')
print('')
print('Paper II FXTs')
for fxt in paper_II_fxts:
    detections = get_detections(str(fxt[0]), fxt[1], fxt[2], True)
    if not detections.empty:
        print(detections[['ObsId', 'RA', 'DEC']].to_string(index=False, header=False))
    elif not analysed[analysed['ObsId'] == str(fxt[0])].empty:
        print(f'{fxt[0]} - not detected')
    else:
        print(f'{fxt[0]} - not analysed')

In [2]:
obsid = 8490
files = glob.glob(f'/data/jcrans/fxrt-data/obsids/{obsid}/s3_expmap_src.fits', recursive=True)
src_file = files[0]

files = glob.glob(f'/data/jcrans/fxrt-data/obsids/{obsid}/*evt2.fits', recursive=True)
event_file = files[0]

with fits.open(src_file) as hdul:
    RA = hdul[1].data['RA']
    RA_err = hdul[1].data['RA_err']
    DEC = hdul[1].data['DEC']
    DEC_err = hdul[1].data['DEC_err']
    X = hdul[1].data['X']
    Y = hdul[1].data['Y']
    X_err = hdul[1].data['X_err']
    Y_err = hdul[1].data['Y_err']
    significance = hdul[1].data['SRC_SIGNIFICANCE']
THETA = []
err_pos = []

t1 = time.perf_counter()

for i, _ in enumerate(RA):
    t_i_1 = time.perf_counter()
 
    THETA.append(
        off_axis(event_file, RA[i], DEC[i])
    )
    err_pos.append(np.sqrt(X_err[i]**2+Y_err[i]**2)*0.492)

    t_i_2 = time.perf_counter()
    print(f"{i+1}/{len(RA)}: {t_i_2-t_i_1:.2f} seconds")

t2 = time.perf_counter()

print(f"off_axis total: {t2-t1:.2f} seconds")
print(THETA)


	proc: 1.03 seconds
1/375: 1.05 seconds
	proc: 1.01 seconds
2/375: 1.03 seconds
	proc: 1.05 seconds
3/375: 1.07 seconds
	proc: 1.06 seconds
4/375: 1.08 seconds
	proc: 0.95 seconds
5/375: 0.96 seconds


KeyboardInterrupt: 

In [8]:


# get time taken for each obsid
first_log = 'logs/redo_everything3.txt'
second_log = 'logs/redo_all_gonnatakeawhile.txt'
first_text = open(first_log, 'r').read()
second_text = open(second_log, 'r').read()

pattern = r"Starting obsid: (\d+)\n.*\n.*\n.*\n\tSearching time: (\d+) seconds\nFinished obsid: \1\nTime: \2 seconds"

matches_1 = re.findall(pattern, first_text)
obsid_search_time_1 = [(int(obsid), int(time)) for obsid, time in matches_1]

matches_2 = re.findall(pattern, second_text)
obsid_search_time_2 = [(int(obsid), int(time)) for obsid, time in matches_2]

difference = [(obsid, time_1, time_2, time_2-time_1) for obsid, time_1 in obsid_search_time_1 for obsid_2, time_2 in obsid_search_time_2 if obsid == obsid_2]

difference

[(86, 1, 4, 3),
 (653, 1, 4, 3),
 (713, 1, 4, 3),
 (767, 1, 4, 3),
 (598, 1, 4, 3),
 (323, 1, 4, 3),
 (324, 0, 4, 4),
 (784, 0, 4, 4),
 (792, 1, 4, 3),
 (800, 1, 4, 3),
 (804, 1, 4, 3),
 (1578, 1, 4, 3),
 (2042, 1, 4, 3),
 (18072, 1, 4, 3),
 (23189, 0, 4, 4),
 (23620, 4, 38, 34),
 (23621, 5, 40, 35),
 (23624, 3, 29, 26),
 (23625, 3, 27, 24),
 (23626, 4, 31, 27),
 (23628, 4, 33, 29),
 (24299, 2, 18, 16),
 (23636, 4, 32, 28),
 (24915, 4, 31, 27),
 (25012, 4, 33, 29),
 (25080, 3, 30, 27),
 (348, 18, 161, 143),
 (352, 23, 205, 182),
 (354, 15, 131, 116),
 (358, 10, 87, 77),
 (361, 1, 4, 3),
 (1302, 0, 4, 4),
 (362, 3, 24, 21),
 (363, 10, 89, 79),
 (367, 8, 74, 66),
 (375, 4, 39, 35),
 (377, 10, 88, 78),
 (379, 10, 86, 76),
 (419, 1, 4, 3),
 (427, 1, 4, 3),
 (434, 2, 15, 13),
 (435, 1, 12, 11),
 (807, 6, 57, 51),
 (808, 7, 66, 59),
 (809, 11, 94, 83),
 (827, 3, 26, 23),
 (830, 23, 208, 185),
 (832, 1, 11, 10),
 (834, 11, 100, 89),
 (835, 1, 4, 3),
 (836, 1, 12, 11),
 (838, 8, 74, 66),
 (839